# Guía del proyecto Estadia / Aprendia Edge Backend

Este notebook documenta el **flujo de trabajo**, **inputs/outputs**, **cómo probar** el API y el **funcionamiento de las métricas** del backend de evaluación automática de caligrafía.

## 1. Flujo de trabajo

El sistema compara el **trazo dibujado por el alumno** (imagen) con una **plantilla de referencia** (esqueleto del carácter) y devuelve una puntuación y métricas.

```
┌─────────────────┐     ┌──────────────────┐     ┌─────────────────┐
│  Imagen PNG/JPG │ ──► │  Preprocesamiento│ ──► │  Esqueleto      │
│  (trazo alumno) │     │  + Esqueletización│     │  del alumno     │
└─────────────────┘     └──────────────────┘     └────────┬────────┘
                                                           │
┌─────────────────┐     ┌──────────────────┐               │
│  Plantilla .npy │ ──► │  Carga plantilla │ ──────────────┤
│  (carácter)     │     │  del carácter    │               │
└─────────────────┘     └──────────────────┘               │
                                                           ▼
┌─────────────────────────────────────────────────────────────────────┐
│  MÉTRICAS                                                            │
│  • Geométricas (IoU, Hausdorff)  • Topología (bucles, puntas)       │
│  • Trayectoria (DTW)              • Calidad (aspect ratio, Hu)       │
└─────────────────────────────────────────────────────────────────────┘
                                                           │
                                                           ▼
┌─────────────────────────────────────────────────────────────────────┐
│  Score final ponderado + imagen de comparación (Base64)              │
└─────────────────────────────────────────────────────────────────────┘
```

### Pasos en detalle

1. **Entrada**: el cliente envía una imagen (trazo del alumno) y el carácter objetivo (`target_char`).
2. **Carga de plantilla**: se carga el esqueleto de referencia (`.npy`) desde `app/templates/`.
3. **Preprocesamiento** (`preprocess_robust` en `processor.py`):
   - Decodificación, binarización adaptativa e inversión (trazo blanco sobre fondo oscuro).
   - Recorte al bounding box, centrado en canvas cuadrado, redimensionado a **256×256**.
   - Suavizado gaussiano y umbral.
   - **Esqueletización** (método Lee) y **poda de espolones** (ramas cortas).
4. **Cálculo de métricas**: comparación patrón vs alumno + calidad solo del alumno.
5. **Score final**: combinación ponderada en `scorer.py`.
6. **Salida**: JSON con `score_final`, métricas e imagen Base64 de comparación.

## 2. Inputs y outputs

### Endpoint: `POST /evaluate`

| Parámetro     | Tipo   | Obligatorio | Descripción |
|---------------|--------|-------------|-------------|
| `file`        | File   | Sí          | Imagen del trazo (PNG, JPG). Formato: multipart/form-data. |
| `target_char` | string | Sí (Form)   | Carácter a evaluar: letra (A–Z, a–z, Ñ, ñ) o dígito (0–9). |

### Respuesta exitosa (200)

```json
{
  "char": "a",
  "score_final": 78.45,
  "metrics": {
    "geometric": { "iou": 62.30, "hausdorff": 8.52, "score": 87.22 },
    "topology": { "match": true, "student": {...}, "pattern": {...} },
    "quality": { "aspect_ratio": 0.85, "shape_fingerprint": [...] },
    "trajectory_error": 4.12
  },
  "image_b64": "iVBORw0KGgoAAAANSUhEUgAA..."
}
```

### Errores

| Código | Condición | Ejemplo |
|--------|-----------|---------|
| 404 | No existe plantilla para el carácter | `"No existe plantilla para 'X'"` |
| 200 | Imagen sin trazo detectable | `{"error": "Sin trazo detectado"}` en el cuerpo |

## 3. Cómo probarlo

### Requisitos

- Python 3.x
- Dependencias: `pip install -r requirements.txt`

### Opción A: Levantar el servidor y usar Swagger

1. En una terminal, desde la raíz del proyecto:
   ```bash
   uvicorn app.main:app --reload
   ```
2. Abre **http://127.0.0.1:8000/docs** (Swagger UI).
3. Expande `POST /evaluate`, pulsa **Try it out**, sube un archivo en `file` y escribe el carácter en `target_char`.
4. Ejecuta y revisa el JSON; puedes decodificar `image_b64` para ver la comparación visual.

### Opción B: Probar con curl

```bash
curl -X POST "http://127.0.0.1:8000/evaluate" \
  -F "file=@ruta/a/tu_imagen_trazo.png" \
  -F "target_char=a"
```

### Opción C: Desde Python (requests)

En la siguiente celda se muestra un ejemplo con `requests` usando una imagen del proyecto.

In [ ]:
# Ejemplo: llamar al API desde Python (el servidor debe estar corriendo en http://127.0.0.1:8000)
import requests

BASE_URL = "http://127.0.0.1:8000"
# Usar una plantilla PNG como "trazo de prueba" (o cualquier imagen con un trazo)
with open("app/templates/a_lower.png", "rb") as f:
    files = {"file": ("a_lower.png", f, "image/png")}
    data = {"target_char": "a"}
    r = requests.post(f"{BASE_URL}/evaluate", files=files, data=data)

if r.status_code == 200 and "error" not in r.json():
    j = r.json()
    print("Carácter:", j["char"])
    print("Score final:", j["score_final"])
    print("Métricas geométricas:", j["metrics"]["geometric"])
    print("Topología (match):", j["metrics"]["topology"]["match"])
else:
    print("Respuesta:", r.status_code, r.json())

### Generar plantillas (opcional)

Si faltan plantillas o quieres regenerarlas:

```bash
python -m app.scripts.generate_templates
```

Genera/sobrescribe `.npy` y `.png` en `app/templates/` para A–Z, Ñ, a–z, ñ, 0–9.

## 4. Funcionamiento de las métricas

Las métricas se calculan en `app/metrics/` y se combinan en `app/metrics/scorer.py`.

### 4.1 Métricas geométricas (`geometric.py`)

Comparan **forma y posición** del esqueleto del alumno frente al de la plantilla.

| Métrica      | Descripción | Interpretación |
|-------------|-------------|----------------|
| **IoU**     | Intersección sobre unión de píxeles del esqueleto (× 100). | Mayor = más solapamiento. 100 = coincidencia perfecta. |
| **hausdorff** | Distancia de Hausdorff (máximo de las dos direcciones) entre conjuntos de puntos de ambos esqueletos. | Menor = formas más parecidas (en píxeles). |
| **score**   | `max(0, 100 - hausdorff * 1.5)`. | Score 0–100 basado en Hausdorff; mayor = mejor. |

- Si los esqueletos tienen distinto tamaño, el del alumno se redimensiona al de la plantilla antes de IoU y Hausdorff.

### 4.2 Topología (`topologic.py`)

Evalúa la **estructura discreta** del trazo: bucles, puntas y cruces.

| Campo        | Descripción |
|-------------|-------------|
| **loops**   | Número de contornos “hijos” (bucles cerrados) detectados en el esqueleto. |
| **endpoints** | Píxeles con exactamente 1 vecino (extremos de ramas). |
| **junctions** | Píxeles con 3 o más vecinos (cruces). |

- **match**: `true` si el número de **loops** del patrón y del alumno coinciden; si no, la estructura de bucles es distinta (ej. "a" con un ojal vs sin ojal).
- En el score final se usa como **todo o nada**: coincidencia de bucles = 100, si no = 30.

### 4.3 Trayectoria (`trajectory.py`)

Aproxima si el **orden/dirección del trazo** se parece al de la plantilla.

- Se obtiene una **secuencia de puntos** del esqueleto ordenando por ángulo respecto al centroide (simulando un trazo radial).
- Se calcula la **distancia media mínima** de cada punto del patrón al punto más cercano del alumno (estilo DTW simplificado).
- **Salida**: un único número en píxeles (`trajectory_error`). Menor = trayectorias más similares; 999 si no hay puntos.

### 4.4 Calidad (`quality.py`)

Métricas **solo del trazo del alumno** (no comparan con la plantilla).

| Métrica            | Descripción |
|--------------------|-------------|
| **aspect_ratio**   | Ancho / alto del bounding box del esqueleto. Útil para ver si la letra está aplastada o estirada. |
| **shape_fingerprint** | Primeros 3 momentos de Hu (transformación logarítmica). Firma geométrica invariante a escala/rotación. |

No entran directamente en el score final; sirven para análisis o feedback adicional.

### 4.5 Score final (`scorer.py`)

Fórmula ponderada que combina:

| Componente   | Peso | Cálculo (resumido) |
|-------------|------|---------------------|
| **Hausdorff** | 45% | `score_h = max(0, 100 - (hausdorff - 10) * 1.8)`. Tolerancia 10 px. |
| **IoU**      | 20% | `score_iou = min(100, iou * 1.5)`. |
| **Topología**| 25% | 100 si `loops` coinciden, 30 si no. |
| **Trayectoria** | 10% | `score_traj = max(0, 100 - trajectory_error * 3)`. |

**Score final** = `0.45*score_h + 0.20*score_iou + 0.25*score_topo + 0.10*score_traj`, redondeado a 2 decimales.

Se prioriza **forma global** (Hausdorff) y **estructura de bucles** (topología), y en menor medida solapamiento (IoU) y fluidez de trazo (trayectoria).

### 4.6 Imagen de comparación (`visualizer.py`)

El campo `image_b64` es una imagen PNG en Base64 que superpone ambos esqueletos:

- **Amarillo**: píxeles donde coinciden plantilla y alumno (acierto).
- **Verde oscuro**: solo plantilla (guía no cubierta).
- **Rojo**: solo alumno (trazo fuera de la plantilla / error).

Título: `Evaluación: {score_final}%`. Tamaño: 256×256.

## 5. Estructura de archivos relevante

```
app/
├── main.py                 # FastAPI app, monta el router
├── api/endpoints.py        # POST /evaluate, orquesta processor + métricas + visualizer
├── core/processor.py       # preprocess_robust, prune_skeleton, esqueletización
├── metrics/
│   ├── geometric.py       # IoU, Hausdorff
│   ├── topologic.py       # bucles, endpoints, junctions
│   ├── trajectory.py      # secuencia por ángulo + distancia media mínima
│   ├── quality.py         # aspect_ratio, Hu moments
│   └── scorer.py          # score final ponderado
├── utils/visualizer.py    # generate_comparison_plot → Base64
├── scripts/generate_templates.py  # generación de plantillas .npy/.png
├── templates/             # plantillas por carácter (ej. a_lower.npy, digit_1.npy)
└── fonts/                 # fuentes para generar plantillas
```